In [1]:
from tensorflow.keras.layers import LSTM,Bidirectional,Input,Concatenate,Dense
from tensorflow.keras.models import Model as Model_


class GloGen_Bidirectional_RNN_Encoder(Model_):
    def __init__(self, num_recurrent_neurons=200):
        super(GloGen_Bidirectional_RNN_Encoder, self).__init__()
        self.encoder = Bidirectional(LSTM(num_recurrent_neurons, return_state=True))
        

    def call(self, input_x):
        self.encoder_outputs, self.forward_h, self.forward_c, self.backward_h, self.backward_c = self.encoder(input_x)
        hidden_state = Concatenate()([self.forward_h, self.backward_h])
        cell_state = Concatenate()([self.forward_c, self.backward_c])
        return [hidden_state,cell_state]
    
class GloGen_Bidirectional_RNN_Decoder(Model_):
    def __init__(self, num_recurrent_neurons=200):
        super(GloGen_Bidirectional_RNN_Decoder, self).__init__()
        self.decoder = LSTM(num_recurrent_neurons*2, return_sequences=True, return_state=True)
        self.decoder_input=Input(shape=(None, num_recurrent_neurons))
        

    def call(self, encoder_hidden_state,num_recurrent_neurons=200):
        self.decoder_outputs, _, _ = self.decoder(self.decoder_input,encoder_hidden_state)
        self.decoder_dense = Dense(num_recurrent_neurons, activation='softmax')
        decoder_outputs = self.decoder_dense(self.decoder_outputs)
        return decoder_outputs

In [2]:
import pandas as pd
import numpy as np
direction_csv_file = pd.read_csv("Directions 1.54138969.csv", header=None)
direction_data_array = np.asarray(direction_csv_file)
direction_data_array = np.reshape(direction_data_array[:100], (10, 10, 64))
direction_data_array.shape

(10, 10, 64)

In [3]:
glogen_bidirectional_RNN_Encoder = GloGen_Bidirectional_RNN_Encoder()
output_encoder = glogen_bidirectional_RNN_Encoder(direction_data_array)
output_encoder


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



[<tf.Tensor: shape=(10, 400), dtype=float32, numpy=
 array([[-0.,  0.,  0., ...,  1.,  0.,  1.],
        [-0.,  0.,  0., ...,  1.,  0.,  1.],
        [-0.,  0.,  0., ...,  1.,  0.,  1.],
        ...,
        [-0., -0.,  0., ...,  1.,  0.,  1.],
        [-0., -0.,  0., ...,  1.,  0.,  1.],
        [-0., -0.,  0., ...,  1.,  0.,  1.]], dtype=float32)>,
 <tf.Tensor: shape=(10, 400), dtype=float32, numpy=
 array([[-1.0000000e+00,  0.0000000e+00,  1.9947275e-07, ...,
          1.0000000e+01,  9.9715602e-01,  1.0000000e+01],
        [-1.0000000e+00,  0.0000000e+00,  2.4670990e-07, ...,
          1.0000000e+01,  9.9875081e-01,  1.0000000e+01],
        [-1.0000000e+00,  0.0000000e+00,  1.2520138e-07, ...,
          1.0000000e+01,  9.9890757e-01,  1.0000000e+01],
        ...,
        [-1.0000000e+00, -8.3269563e-35,  2.5784844e-10, ...,
          1.0000000e+01,  3.9747858e-05,  1.0000000e+01],
        [-1.0000000e+00, -3.3250445e-35,  3.3862424e-10, ...,
          1.0000000e+01,  6.3676011e-06,

In [4]:
glogen_decoder = GloGen_Bidirectional_RNN_Decoder()
glogen_sparse_output_predictions = glogen_decoder(output_encoder)
print(glogen_sparse_output_predictions.shape)
glogen_sparse_output_predictions

(10, None, 200)


<tf.Tensor 'dense/truediv:0' shape=(10, None, 200) dtype=float32>